In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


### Data Load

In [ ]:
movies = pd.read_csv('/gdrive/MyDrive/playdata/movie_recommendation/data/movies.csv')
movies.head(2)

,movie_id,title,release_date,imdb_url,all_genres
0,0,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,Animation-Children-Comedy
1,1,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,Action-Adventure-Thriller


In [ ]:
users = pd.read_csv('/gdrive/MyDrive/playdata/movie_recommendation/data/users_update.csv')
users.tail()

,user_id,age,sex,occupation
939,939,32,M,administrator
940,940,20,M,student
941,941,48,F,librarian
942,942,22,M,student
943,943,25,M,heonter


In [ ]:
ratings = pd.read_csv('/gdrive/MyDrive/playdata/movie_recommendation/data/ratings_update.csv')
ratings.tail()

,rating_id,user_id,movie_id,rating
100005,100006,943,285,1
100006,100007,943,287,1
100007,100008,943,0,4
100008,100009,943,299,4
100009,100010,943,120,5


- train, test set

In [ ]:
# df를 train/test set으로 분할하는 함수 생성
# Args: df, holdoyt_fraction(분할 비율)
def split_dataframe(df, holdout_fraction=0.1):
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]

  return train, test

In [ ]:
ratings['user_id'] = ratings['user_id'].astype(int)
ratings['movie_id'] = ratings['movie_id'].astype(int)

train_ratings, test_ratings = split_dataframe(ratings)
train_ratings.shape, test_ratings.shape

((90009, 4), (10001, 4))

### Matrix Factorization

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, dot

- Model 생성

In [ ]:
users.shape[0], movies.shape[0]

(944, 1682)

In [ ]:
n_latent_factors = 30 # user와 movie embedding의 차원수

user_input = Input(shape=[1], name='user')
movie_input = Input(shape=[1], name='movie')

user_embedding = Embedding(input_dim=users.shape[0] # 944
                           , output_dim = n_latent_factors # 30
                           , name='user_embedding'
                           )(user_input)

movie_embedding = Embedding(input_dim=movies.shape[0] # 1682
                           , output_dim = n_latent_factors # 30
                           , name='movie_embedding'
                           )(movie_input)

user_vec = Flatten(name='flatten_users')(user_embedding) # 1차원 배열로 변환
movie_vec = Flatten(name='flatten_movies')(movie_embedding) # 1차원 배열로 변환

product = dot([movie_vec, user_vec], axes=1) # 평점
model = Model(inputs=[user_input, movie_input], outputs=product) # user와 movie가 주어졌을 때, 평점을 예측하는 모델

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 movie (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 user (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 movie_embedding (Embedding)    (None, 1, 30)        50460       ['movie[0][0]']                  
                                                                                                  
 user_embedding (Embedding)     (None, 1, 30)        28320       ['user[0][0]']                   
                                                                                              

- compile

In [ ]:
model.compile(optimizer='adam', loss='mse')

- fit

In [ ]:
history = model.fit(x=[train_ratings['user_id'], train_ratings['movie_id']],
                    y=train_ratings['rating'], epochs=500,
                    validation_data=([test_ratings['user_id'], test_ratings['movie_id'] ],
                                     test_ratings['rating']),
                    verbose=1, batch_size=train_ratings.shape[0])

Epoch 1/500
1/1 [==============================] - 6s 6s/step - loss: 13.7145 - val_loss: 13.8366
Epoch 2/500
1/1 [==============================] - 0s 61ms/step - loss: 13.7136 - val_loss: 13.8366
Epoch 3/500
1/1 [==============================] - 0s 67ms/step - loss: 13.7127 - val_loss: 13.8366
Epoch 4/500
1/1 [==============================] - 0s 51ms/step - loss: 13.7118 - val_loss: 13.8364
Epoch 5/500
1/1 [==============================] - 0s 53ms/step - loss: 13.7108 - val_loss: 13.8362
Epoch 6/500
1/1 [==============================] - 0s 53ms/step - loss: 13.7096 - val_loss: 13.8359
Epoch 7/500
1/1 [==============================] - 0s 53ms/step - loss: 13.7084 - val_loss: 13.8354
Epoch 8/500
1/1 [==============================] - 0s 35ms/step - loss: 13.7070 - val_loss: 13.8347
Epoch 9/500
1/1 [==============================] - 0s 41ms/step - loss: 13.7055 - val_loss: 13.8338
Epoch 10/500
1/1 [==============================] - 0s 40ms/step - loss: 13.7038 - val_loss: 13.8326
E

In [ ]:
# import pickle
# with open('CFModel_01.pickle','wb') as fw:
#     pickle.dump(model, fw)

In [ ]:
# with open('CFModel_01.pickle', 'rb') as f:
#     CFModel_01 = pickle.load(f)

In [ ]:
model.save('CFModel02.h5')

In [ ]:
import tensorflow as tf
CFModel02 = tf.keras.models.load_model('CFModel02.h5')
CFModel02.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 movie (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 user (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 movie_embedding (Embedding)    (None, 1, 30)        50460       ['movie[0][0]']                  
                                                                                                  
 user_embedding (Embedding)     (None, 1, 30)        28320       ['user[0][0]']                   
                                                                                              